# Data

In [1]:
import glob
import os
import pandas as pd
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
# pd.reset_option('all')

# data_dir = "/home/ec2-user/SageMaker/efs/Projects/Qwen2-VL/data/快递视频1225"
data_dir = "/home/ec2-user/SageMaker/efs/Projects/LLaVA-NeXT/data/hualai_sft_data"
video_dir = "/home/ec2-user/SageMaker/efs/Projects/LLaVA-NeXT/data/hualai_sft_data/video"

# video_files = sorted(glob.glob(os.path.join(data_dir, '*.mp4')) + glob.glob(os.path.join(data_dir, '*.mov')), key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))
# print(video_files)

# label_path = os.path.join(data_dir, 'description.xlsx')
# df_test = pd.read_excel(label_path)
# df_test.head()

test_path = os.path.join(data_dir, 'test.json')
with open(test_path, 'r') as f:
    test_data = json.load(f)
test_list = [v for k, v in test_data.items()]
df_test = pd.DataFrame(test_list)
df_test['id'] = df_test['video_id'].apply(lambda x: int(x.split('-')[1].split('.')[0]))
df_test.sort_values(by=['id'], inplace=True)
df_test = df_test.reset_index(drop=True)
df_test.head()


,scenario,delivery_service,description,video_id,id
0,Normal express delivery,Unknown,"The courier, wearing a black t-shirt with design and a baseball cap, is delivering a 'CAVA' branded bag. The courier is knocking on the door and attempting to hand off the package",anonymized-4.mp4,4
1,Normal express delivery,Unknown,A person in a red jacket drops off a paper bag at the front door. After the person droped the paper bag the dog in the house got out.,anonymized-5.mp4,5
2,Normal express delivery,Amazon,An Amazon delivery driver throws a package onto the porch.,anonymized-12.mp4,12
3,Normal express delivery,Amazon,An Amazon delivery driver is dropping off a package at the front door of a residence.,anonymized-14.mp4,14
4,Violent express delivery,Unknown,Two couriers are carrying a large package. One of the couriers throws the package onto the ground.,anonymized-15.mp4,15


In [2]:
# import os
# import pandas as pd
# from decord import VideoReader

# # Function to get video duration using Decord
# def get_video_duration_decord(video_path):
#     try:
#         # Open the video file
#         vr = VideoReader(video_path)
        
#         # Get frame count and fps
#         frame_count = len(vr)
#         fps = vr.get_avg_fps()
        
#         # Calculate duration in seconds
#         duration = frame_count / fps
        
#         return round(duration, 2)
#     except Exception as e:
#         print(f"Error processing {video_path}: {e}")
#         return None

# # Add duration column to dataframe
# def add_duration_to_df(df, video_folder):
#     # Create a new column for duration
#     df['duration'] = 0
    
#     # Process each video and add duration to dataframe
#     for index, row in df.iterrows():
#         video_id = row['video_id']
#         video_path = os.path.join(video_folder, video_id)
        
#         if os.path.exists(video_path):
#             duration = get_video_duration_decord(video_path)
#             df.at[index, 'duration'] = duration
#         else:
#             print(f"Warning: Video file {video_path} not found")
    
#     return df

# # Usage
# df_test = add_duration_to_df(df_test, video_dir)
# # df_test.head(10)
# df_test[['duration']].describe(percentiles=[0.5, 0.6, 0.8, 0.9, 0.95, 0.99])

# LLaVA-Video

In [3]:
# pip install git+https://github.com/LLaVA-VL/LLaVA-NeXT.git
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path, process_images, tokenizer_image_token
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN, IGNORE_INDEX
from llava.conversation import conv_templates, SeparatorStyle
from PIL import Image
import requests
import copy
import torch
import sys
import warnings
from decord import VideoReader, cpu
import numpy as np
warnings.filterwarnings("ignore")
def load_video(video_path, max_frames_num,fps=1,force_sample=False):
    if max_frames_num == 0:
        return np.zeros((1, 336, 336, 3))
    vr = VideoReader(video_path, ctx=cpu(0),num_threads=1)
    total_frame_num = len(vr)
    video_time = total_frame_num / vr.get_avg_fps()
    fps = round(vr.get_avg_fps()/fps)
    frame_idx = [i for i in range(0, len(vr), fps)]
    frame_time = [i/fps for i in frame_idx]
    if len(frame_idx) > max_frames_num or force_sample:
        sample_fps = max_frames_num
        uniform_sampled_frames = np.linspace(0, total_frame_num - 1, sample_fps, dtype=int)
        frame_idx = uniform_sampled_frames.tolist()
        frame_time = [i/vr.get_avg_fps() for i in frame_idx]
    frame_time = ",".join([f"{i:.2f}s" for i in frame_time])
    spare_frames = vr.get_batch(frame_idx).asnumpy()
    # import pdb;pdb.set_trace()
    return spare_frames,frame_time,video_time


/home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ec2-user/SageMaker/efs/conda_envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:

# pretrained = "lmms-lab/LLaVA-Video-7B-Qwen2"
# model_dir = "/home/ec2-user/SageMaker/efs/Models/LLaVA-Video-7B-Qwen2"
# model_dir = "/home/ec2-user/SageMaker/efs/Projects/LLaVA-NeXT/checkpoints/llavanext-google_siglip-so400m-patch14-384-Qwen_Qwen2-7B-Instruct-ov_to_video_am9_aug17"
# model_dir = "/home/ec2-user/SageMaker/efs/Models/LLaVA-Video-72B-Qwen2"
# model_name = model_dir.rsplit('/', 1)[1]

ckpt = 250
model_dir = f"/home/ec2-user/SageMaker/efs/Projects/LLaVA-NeXT/checkpoints/llava-video-task-full-2025-03-19-05-52-43/checkpoint-{ckpt}"
model_name = "LLaVA-Video-7B-Qwen2"

device = "cuda"
device_map = "auto"
tokenizer, model, image_processor, max_length = load_pretrained_model(model_dir, None, model_name, torch_dtype="bfloat16", device_map=device_map)  # Add any other thing you want to pass in llava_model_args
model.eval()

Loaded LLaVA model: /home/ec2-user/SageMaker/efs/Projects/LLaVA-NeXT/checkpoints/llava-video-task-full-2025-03-19-05-52-43/checkpoint-250


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type llava to instantiate a model of type llava_qwen. This is not supported for all configurations of models and can yield errors.


Loading vision tower: google/siglip-so400m-patch14-384


Loading checkpoint shards: 100%|██████████| 4/4 [02:47<00:00, 41.76s/it]

Model Class: LlavaQwenForCausalLM


LlavaQwenForCausalLM(
  (model): LlavaQwenModel(
    (embed_tokens): Embedding(151647, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2FlashAttention2(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
    (visio

In [5]:
import json

# def convert_to_dict(text_item):
#     if text_item.startswith('```'):
#         # Handle markdown code block format
#         # Remove ```json\n at start and \n``` at end
#         json_str = text_item.replace('```json\n', '').replace('\n```', '')
#     else:
#         # Handle double curly brace format
#         json_str = text_item.replace('{{', '{').replace('}}', '}')
    
#     return json.loads(json_str)

def convert_to_dict(text_item):
    # Strip any leading/trailing whitespace
    text_item = text_item.strip()
    
    # Remove outer quotes if present (like in your example)
    if (text_item.startswith("'") and text_item.endswith("'")) or \
       (text_item.startswith('"') and text_item.endswith('"')):
        text_item = text_item[1:-1]
    
    # Handle markdown code blocks or double curly braces
    if text_item.startswith('```'):
        json_str = text_item.replace('```json\n', '').replace('\n```', '')
    else:
        json_str = text_item.replace('{{', '{').replace('}}', '}')
    
    try:
        # Try parsing directly first
        return json.loads(json_str)
    except json.JSONDecodeError:
        # If that fails, try using Python's ast.literal_eval
        # which handles some quote issues better
        try:
            import ast
            return ast.literal_eval(json_str)
        except:
            # If that fails too, look for quoted speech in the text
            # and escape those quotes
            import re
            
            # This pattern looks for text like: says: "Something"
            # and escapes the inner quotes
            fixed_json = re.sub(r'(says:\s*)"([^"]+)"', r'\1\\"\\2\\"', json_str)
            
            try:
                return json.loads(fixed_json)
            except:
                # If specific fix didn't work, try a more general approach
                # This pattern finds any unescaped quotes in description field
                if '"description"' in json_str:
                    desc_start = json_str.find('"description"')
                    if desc_start >= 0:
                        # Find beginning and end of the description value
                        value_start = json_str.find('": "', desc_start) + 4
                        
                        # Look for the end - first quote followed by comma or closing brace
                        for i in range(value_start, len(json_str)):
                            if json_str[i] == '"' and i+1 < len(json_str) and json_str[i+1] in [',', '}']:
                                # Extract and fix the description
                                desc = json_str[value_start:i]
                                fixed_desc = desc.replace('"', '\\"')
                                fixed_json = json_str[:value_start] + fixed_desc + json_str[i:]
                                return json.loads(fixed_json)
                
                # If all else fails, raise a clear error
                raise ValueError(f"Could not parse JSON: {json_str}")

def identify_event(video_path, fps=1, max_frames_num = 32):
#     prompt = """\
# This is the surveillance camera footage from home delivery scenarios. I need you to analyze the content carefully and provide a structured analysis in the following format:

# scenario: identify the main events in the video, select the most relevant label in [Deliver a package, The package was stolen, Damaging a package violently, The delivery driver approached, Take the box, Violently delivering packages, Place the box, Deliver food, Driving a delivery van]
# delivery service: identify the delivery company if visible, e.g., FedEx, UPS, Amazon, 美团外卖, SAGAWA, DHL, or mark as “Unknown” if unclear
# description: provide a brief, one-sentence description of what happens in the video, including key actions and participants ** in English **

# Return your analysis in strict JSON format as follows:
# {{
#     "scenario": "main event or situation labels",
#     "delivery_service": "company name or Unknown",
#     "description": "brief one-sentence description in English",
# }}
# Please ensure the output is valid JSON with these exact keys.
# """

    prompt = """\
This is the surveillance camera footage from home delivery scenarios. I need you to analyze the content carefully and provide a structured analysis in the following format:

scenario: identify the main events in the video, select the most relevant label in [Violent express delivery, Normal express delivery, Express package theft, Encounter with wildlife, Accidental dropping of delivery items, Other abnormal behaviors]
delivery service: identify the delivery company if visible, e.g., FedEx, UPS, Amazon, 美团外卖, SAGAWA, DHL, or mark as “Unknown” if unclear
description: provide a brief, one-sentence description of what happens in the video, including key actions and participants ** in English **

Return your analysis in strict JSON format as follows:
{{
    "scenario": "main event or situation labels",
    "delivery_service": "company name or Unknown",
    "description": "brief one-sentence description in English",
}}
Please ensure the output is valid JSON with these exact keys.
"""
    # max_frames_num = 32
    video,frame_time,video_time = load_video(video_path, max_frames_num, fps, force_sample=True)
    video = image_processor.preprocess(video, return_tensors="pt")["pixel_values"].cuda().bfloat16() # .half()
    video = [video]
    conv_template = "qwen_1_5"  # Make sure you use correct chat template for different models
    time_instruciton = f"The video lasts for {video_time:.2f} seconds, and {len(video[0])} frames are uniformly sampled from it. These frames are located at {frame_time}.Please answer the following questions related to this video."
    question = DEFAULT_IMAGE_TOKEN + f"\n{time_instruciton}\n{prompt}" # Please describe this video in detail."
    conv = copy.deepcopy(conv_templates[conv_template])
    conv.append_message(conv.roles[0], question)
    conv.append_message(conv.roles[1], None)
    prompt_question = conv.get_prompt()
    # print(f"prompt_question: {prompt_question}")
    input_ids = tokenizer_image_token(prompt_question, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt").unsqueeze(0).to(device)
    cont = model.generate(
        input_ids,
        images=video,
        modalities= ["video"],
        do_sample=False,
        temperature=0,
        max_new_tokens=4096,
    )
    text_outputs = tokenizer.batch_decode(cont, skip_special_tokens=True)# [0].strip()
    # print(text_outputs)
    result = convert_to_dict(text_outputs[0])
    
    return result

sample_path = os.path.join(video_dir, df_test.loc[10, 'video_id'])
result = identify_event(sample_path)
print(f"result: {result}")

result: {'scenario': 'Normal express delivery', 'delivery_service': 'Unknown', 'description': "A Tesco delivery driver is delivering a package to a house. The driver says, 'No, this is my house' while holding the package."}


In [6]:
from tqdm import tqdm
import time

model_name = model_dir.rsplit('/', 1)[1]
fps = 1
max_frames_num = 32

results = []
for i, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    start_time = time.time()
    video_file = os.path.join(video_dir, row['video_id'])
    try:
        result = identify_event(video_file, fps, max_frames_num)
    except Exception as e:
        print(f"Error: {e}")
        print(f"result: {result}")
        
    time_cost = time.time() - start_time
    result['time_cost'] = round(time_cost, 3)
    results.append(result)
    
df_pred = pd.DataFrame(results)
df_pred.to_csv(f"./outputs/result_ckpt{ckpt}_{fps}_{max_frames_num}.csv", index=False)
df_pred.head()

100%|██████████| 21/21 [01:25<00:00,  4.06s/it]


,scenario,delivery_service,description,time_cost
0,Normal express delivery,Unknown,"A delivery person wearing a black shirt with yellow text and a cap delivers a package to the front door of a building. The delivery person appears to be in a hurry, as indicated by their quick approach to the door and the way they place the package there.",5.729
1,Violent express delivery,Unknown,"A person in a red jacket throws a package onto the porch, causing the security camera to shake.",3.423
2,Normal express delivery,Amazon,An Amazon delivery driver attempts a forward roll on the lawn after delivering a package but fails and falls to the ground.,3.431
3,Normal express delivery,Amazon,An Amazon delivery driver is delivering a package to a house. The driver greets the customer and places the package at the front door.,3.492
4,Violent express delivery,Unknown,A person in a blue shirt is seen dropping a package onto the ground in an outdoor corridor with brick walls.,3.306


# evaluate results

In [7]:
# rename_dict = {
#     'scenario_pred_pred': 'scenario_pred', 
#     'delivery_service_pred_pred': 'delivery_service_pred',
#     'description_pred_pred': 'description_pred',
#     'time_cost_pred_pred': 'time_cost_pred'
# }

# # Rename the columns
# df_pred = df_pred.rename(columns=rename_dict)

In [8]:
df_pred = df_pred.add_suffix(f'_pred')
df_merge = pd.concat([df_test, df_pred], axis=1)
columns = ['video_id', 'scenario', 'scenario_pred', 'delivery_service', 'delivery_service_pred', 
           'description', 'description_pred', 'time_cost_pred']
df_merge = df_merge[columns]
df_merge.head()

,video_id,scenario,scenario_pred,delivery_service,delivery_service_pred,description,description_pred,time_cost_pred
0,anonymized-4.mp4,Normal express delivery,Normal express delivery,Unknown,Unknown,"The courier, wearing a black t-shirt with design and a baseball cap, is delivering a 'CAVA' branded bag. The courier is knocking on the door and attempting to hand off the package","A delivery person wearing a black shirt with yellow text and a cap delivers a package to the front door of a building. The delivery person appears to be in a hurry, as indicated by their quick approach to the door and the way they place the package there.",5.729
1,anonymized-5.mp4,Normal express delivery,Violent express delivery,Unknown,Unknown,A person in a red jacket drops off a paper bag at the front door. After the person droped the paper bag the dog in the house got out.,"A person in a red jacket throws a package onto the porch, causing the security camera to shake.",3.423
2,anonymized-12.mp4,Normal express delivery,Normal express delivery,Amazon,Amazon,An Amazon delivery driver throws a package onto the porch.,An Amazon delivery driver attempts a forward roll on the lawn after delivering a package but fails and falls to the ground.,3.431
3,anonymized-14.mp4,Normal express delivery,Normal express delivery,Amazon,Amazon,An Amazon delivery driver is dropping off a package at the front door of a residence.,An Amazon delivery driver is delivering a package to a house. The driver greets the customer and places the package at the front door.,3.492
4,anonymized-15.mp4,Violent express delivery,Violent express delivery,Unknown,Unknown,Two couriers are carrying a large package. One of the couriers throws the package onto the ground.,A person in a blue shirt is seen dropping a package onto the ground in an outdoor corridor with brick walls.,3.306


In [9]:
# !pip install rouge
# !pip install nltk

In [10]:
from rouge import Rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# df_merge['scenario_correct'] = (df_merge['scenario'] == df_merge['scenario_pred']).astype(int)
# df_merge['delivery_service_correct'] = (df_merge['delivery_service'] == df_merge['delivery_service_pred']).astype(int)

df_merge['scenario_correct'] = (df_merge['scenario'].str.lower() == df_merge['scenario_pred'].str.lower()).astype(int)
df_merge['delivery_service_correct'] = (df_merge['delivery_service'].str.lower() == df_merge['delivery_service_pred'].str.lower()).astype(int)

bleu_smoothing = SmoothingFunction().method4
rouge = Rouge()

for index, row in df_merge.iterrows():
    reference = row['description']
    hypothesis = row['description_pred']

    df_merge.at[index, 'description_bleu'] = sentence_bleu([reference], hypothesis, smoothing_function=bleu_smoothing)
    rouge_scores = rouge.get_scores(hypothesis, reference)[0]
    df_merge.at[index, 'description_rouge'] = rouge_scores['rouge-l']['f']

df_merge.to_csv(f"outputs/merge_result_ckpt{ckpt}_{fps}_{max_frames_num}.csv", index=False)
df_merge.head()

,video_id,scenario,scenario_pred,delivery_service,delivery_service_pred,description,description_pred,time_cost_pred,scenario_correct,delivery_service_correct,description_bleu,description_rouge
0,anonymized-4.mp4,Normal express delivery,Normal express delivery,Unknown,Unknown,"The courier, wearing a black t-shirt with design and a baseball cap, is delivering a 'CAVA' branded bag. The courier is knocking on the door and attempting to hand off the package","A delivery person wearing a black shirt with yellow text and a cap delivers a package to the front door of a building. The delivery person appears to be in a hurry, as indicated by their quick approach to the door and the way they place the package there.",5.729,1,1,0.379113,0.327869
1,anonymized-5.mp4,Normal express delivery,Violent express delivery,Unknown,Unknown,A person in a red jacket drops off a paper bag at the front door. After the person droped the paper bag the dog in the house got out.,"A person in a red jacket throws a package onto the porch, causing the security camera to shake.",3.423,0,1,0.337740,0.388889
2,anonymized-12.mp4,Normal express delivery,Normal express delivery,Amazon,Amazon,An Amazon delivery driver throws a package onto the porch.,An Amazon delivery driver attempts a forward roll on the lawn after delivering a package but fails and falls to the ground.,3.431,1,1,0.367342,0.466667
3,anonymized-14.mp4,Normal express delivery,Normal express delivery,Amazon,Amazon,An Amazon delivery driver is dropping off a package at the front door of a residence.,An Amazon delivery driver is delivering a package to a house. The driver greets the customer and places the package at the front door.,3.492,1,1,0.485631,0.647059
4,anonymized-15.mp4,Violent express delivery,Violent express delivery,Unknown,Unknown,Two couriers are carrying a large package. One of the couriers throws the package onto the ground.,A person in a blue shirt is seen dropping a package onto the ground in an outdoor corridor with brick walls.,3.306,1,1,0.369404,0.312500


In [11]:
df_eval = df_merge[['scenario_correct', 'delivery_service_correct', 'description_bleu', 'description_rouge']].mean().round(4)
df_eval.to_csv(f"outputs/eval_result_ckpt{ckpt}_{fps}_{max_frames_num}.csv")
df_eval

scenario_correct            0.5714
delivery_service_correct    0.7619
description_bleu            0.3151
description_rouge           0.3669
dtype: float64

# Evaluate using SGLang

In [12]:
from openai import OpenAI

port = 30008
model_name = "/home/ec2-user/SageMaker/efs/Models/LLaVA-Video-7B-Qwen2"
client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://raw.githubusercontent.com/sgl-project/sglang/main/assets/logo.png",
                    },
                },
                {
                    "type": "text",
                    "text": "I have two very different images. They are not related at all. "
                    "Please describe the first image in one sentence, and then describe the second image in another sentence.",
                },
            ],
        }
    ]

messages = [
    {
        "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "./data/piechart-QA/piechart-QA/image_1.png"
                    }
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "./data/piechart-QA/piechart-QA/image_2.png"
                    }
                },
                {
                    "type": "text",
                    "text": "What's the difference between the above two images?"
                }
            ]
    }
]

response = client.chat.completions.create(
    model=model_name,
    messages=messages,
    temperature=0,
    stream=True
)

# print(response.choices[0].message.content)

# Handle the streaming output
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

APIConnectionError: Connection error.

In [ ]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

# Use stream=True for streaming responses
response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
    stream=True,
)

# Handle the streaming output
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

In [ ]:
import json
import base64
import numpy as np
import io

# Function to encode the image
def encode_image(image):
    if isinstance(image, str):
        with open(image, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")
    elif isinstance(image, bytes):
        return base64.b64encode(image).decode("utf-8")
    elif isinstance(image, np.ndarray):
        # Convert numpy array to PIL Image
        image = Image.fromarray(np.uint8(image))
        # Create a buffer to hold the image data
        buffer = io.BytesIO()
        # Save the image to the buffer in PNG format
        image.save(buffer, format="PNG")
        # Get the binary data from the buffer
        img_bytes = buffer.getvalue()
        return base64.b64encode(img_bytes).decode("utf-8")
    else:
        raise ValueError("The input should be either a file path (string), binary image data, or a numpy array.")

def convert_to_dict(text_item):
    if text_item.startswith('```'):
        # Handle markdown code block format
        # Remove ```json\n at start and \n``` at end
        json_str = text_item.replace('```json\n', '').replace('\n```', '')
    else:
        # Handle double curly brace format
        json_str = text_item.replace('{{', '{').replace('}}', '}')
    
    return json.loads(json_str)

def identify_event_sglang(video_path, prompt=None):
    if prompt is None:
        prompt = """\
This is the surveillance camera footage from home delivery scenarios. I need you to analyze the content carefully and provide a structured analysis in the following format:

scenario: identify the main events in the video, select the most relevant label in [Deliver a package, The package was stolen, Damaging a package violently, The delivery driver approached, Take the box, Violently delivering packages, Place the box, Deliver food, Driving a delivery van]
delivery service: identify the delivery company if visible, e.g., FedEx, UPS, Amazon, 美团外卖, SAGAWA, DHL, or mark as “Unknown” if unclear
description: provide a brief, one-sentence description of what happens in the video, including key actions and participants ** in English **

Return your analysis in strict JSON format as follows:
{{
    "scenario": "main event or situation labels",
    "delivery_service": "company name or Unknown",
    "description": "brief one-sentence description in English",
}}
Please ensure the output is valid JSON with these exact keys.
"""

    max_frames_num = 32
    video, frame_time, video_time = load_video(video_path, max_frames_num, 1, force_sample=True)
    content = []
    for image in video:
        base64_image = encode_image(image)
        content.append(
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                }
            }
        )
    
    content.append(
        {
            "type": "text",
            "text": prompt
        }
    )

    messages = [
        {
             "role": "user",
             "content": content
        }
    ]

    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        temperature=0,
        # stream=True
    )

    return response.choices[0].message.content
    
result = identify_event_sglang(video_files[1])
print(f"result: {result}")

In [ ]:
# from tqdm import tqdm
# import time

# model_name = model_dir.rsplit('/', 1)[1]

# results = []
# for video_file in tqdm(video_files, total=len(video_files)):
#     start_time = time.time()
#     try:
#         result = identify_event(video_file)
#     except Exception as e:
#         print(f"Error: {e}")
#         print(f"result: {result}")
        
#     time_cost = time.time() - start_time
#     result['time_cost'] = round(time_cost, 3)
#     results.append(result)
    
# df_pred = pd.DataFrame(results)
# df_pred.to_csv(f"./outputs/result_{model_name}.csv", index=False)
# df_pred.head()

In [ ]:
import json

data_folder = "./data/0_30_s_activitynetqa"
data_path = "./data/0_30_s_activitynetqa/0_30_s_activitynetqa_oe_qa_processed.json"
with open(data_path, 'r') as f:
    data = json.load(f)

data[:3]

In [ ]:
from tqdm import tqdm
import time

model_name = model_dir.rsplit('/', 1)[1]

results = []
for row in tqdm(data[:100], total=len(data)):
    prompt = row['conversations'][0]['value'].replace('<image>\n', '')
    video_path = os.path.join(data_folder, row['video'])
    start_time = time.time()
    result = {
        'prompt': prompt,
        'label': row['conversations'][1]['value'],
    }
    # print(f"row: {row}")
    try:
        response = identify_event_sglang(video_path, prompt)
        result['predict'] = response
        # print(f"result: {result}")
    except Exception as e:
        print(f"Error: {e}")
        print(f"result: {result}")
        
    time_cost = time.time() - start_time
    result['time_cost'] = round(time_cost, 3)
    result['video'] = video_path
    results.append(result)
    
df_pred = pd.DataFrame(results)
df_pred.to_csv(f"./outputs/result_{model_name}.csv", index=False)
df_pred.head()

In [ ]:
df_pred['correct'] = (df_pred['predict'] == df_pred['label']).astype(int)
df_pred.head()

In [ ]:
accuracy = df_pred['correct'].sum() / df_pred.shape[0]
print(f"accuracy: {accuracy:.2f}")